In [111]:
import nltk
import itertools
import operator
import pandas as pd
import numpy as np
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\deept\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
reviews = pd.read_csv('yelp__review.csv', low_memory=False)

In [101]:
reviews[0:2]

,Unnamed: 0,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,0,dDl8zu1vWPdKGihJrwQbpw,msQe1u7Z_XuqjGoqhB0J5g,pomGBqfbxcqPv14c3XH-ZQ,5.0,2012-11-13,I love this place! My fiance And I go here atl...,0.0,0.0,0.0
1,0,LZp4UX5zK3e-c5ZGSeo3kA,msQe1u7Z_XuqjGoqhB0J5g,jtQARsP6P-LbkyjbO1qNGg,1.0,2014-10-23,Terrible. Dry corn bread. Rib tips were all fa...,3.0,1.0,1.0


In [105]:
reviews[0:1]['text']

0    I love this place! My fiance And I go here atl...
Name: text, dtype: object

In [ ]:
reviews_df = reviews[['business_id','text']]

In [113]:
reviews_df.head()

,business_id,text
0,pomGBqfbxcqPv14c3XH-ZQ,I love this place! My fiance And I go here atl...
1,jtQARsP6P-LbkyjbO1qNGg,Terrible. Dry corn bread. Rib tips were all fa...
2,elqbBhBfElMNSrjFqW3now,Back in 2005-2007 this place was my FAVORITE t...
3,Ums3gaP2qM3W1XcA5r6SsQ,Delicious healthy food. The steak is amazing. ...
4,vgfcTvK81oD4r50NMjU2Ag,This place sucks. The customer service is horr...


In [109]:
reviews_df.count()

business_id    5996997
text           5996994
dtype: int64

In [ ]:
sentiment_analyzer = SentimentIntensityAnalyzer()

In [125]:
df = reviews_df.groupby('business_id')['text'].nunique()
print(df)

business_id
--1UhMGODdWsrMastO9DZw      24
--6MefnULPED_I942VcFNA      39
--7zmmkVg-IMGaXbuVd0SQ      54
--8LPVSo5i0Oo61X01sV9A       4
--9QQLMTbFzLJ_oT-ON3Xw      11
--9e1ONYQuAa-CB_Rrw7Tw    1545
--DaPTJW3-tB1vP-PfdTEg      45
--DdmeR16TRb3LsjG0ejrQ       5
--EF5N7P70J_UYBTPypYlA       5
--EX4rRznJrltyn-34Jz1w       4
--FBCX-N37CMYDfs790Bnw     125
--FLdgM0GNpXVMn74ppCGw      12
--GM_ORV2cYS-h38DSaCLw       8
--Gc998IMjLn8yr-HTzGUg       3
--I7YYLada0tSLkORTHb5Q      83
--KCl2FvVQpvjzmZSPyviA      13
--KQsXc-clkO7oHRqGzSzg      35
--Ni3oJ4VOqfOEu7Sj2Vzg       6
--Rsj71PBe31h5YljVseKA       8
--S62v0QgkqQaVUhFnNHrw      35
--SrzpvFLwP_YFwB_Cetow      44
--TcDRzRIxhvHM4DSgEuMA       5
--U98MNlDym2cLn36BBPgQ       4
--VMPfs4zfZJtQbqzJsNhg      10
--WsruI0IGEoeRmkErU5Gg      15
--Y7NhBKzLTbNliMUX_wfg      12
--YPwqIlRJrhHkJcjY3eiA       3
--ab39IjZR_xUf81WyTyHg      11
--cZ6Hhc9F7VkKXxHMVZSQ     333
--cgVkbWTiga3OYTkymKqA      13
                          ... 
zzXuJF6UUT1kgTyCsmgDmQ     

In [119]:
dict_ = reviews_df.set_index('business_id')['text'].to_dict()

In [120]:
dict_

{'text': {'pomGBqfbxcqPv14c3XH-ZQ': 'We LOVED Cantina Laredo before it became Leticia\'s so we were hoping for the same wonderful Mexican food.  It started out quite well with tasty salsa and bean dip but the goodness ended there.  My husband and I ordered the chicken fajitas.  Ugh!!! Worst fajitas ever!!  They arrived on the usual black grill plate only they weren\'t steaming....lukewarm at best in a soupy juice of zucchini and tiny little bits of "cooked chicken" not even grilled.  It reminded me of the chicken you get in chow mein.  There was absolutely no Mexican flavor.  We were so disappointed and will not be returning.',
  'jtQARsP6P-LbkyjbO1qNGg': 'Tried this place a few times, the ribs are excellent, the okras are pleasing, I love the black eyes peas except I find it always is a bit salty! I actually enjoy their food however I might find myself with high blood pressure if I continue to eat here. So my negative is that they ought to tone the level of salt down a bit! I recommen

In [89]:
count = 0
upper_limit = 0
lower_limit = 0
sent = {}
while count!=170000:
    lower_limit=upper_limit
    upper_limit+=10000
    x = itertools.islice(dict_.items(), lower_limit,upper_limit)
    for key,val in x:
        count += 1
        if key not in sent:
            sent[key] = []
        if type(val) == 'float':
            print(key)
        sentiment = sentiment_analyzer.polarity_scores(val)
#         if len(val) == 1:
#             sentiment = sentiment_analyzer.polarity_scores(val)
#         else:
#             sentiment = sentiment_analyzer.polarity_scores(val[0])
        sentiment.pop('compound')
        sentiment.pop('neu')
        sent[key].append(max(sentiment.items(), key=operator.itemgetter(1))[0])
    print(lower_limit, upper_limit, count)

0 10000 10000
10000 20000 20000
20000 30000 30000
30000 40000 40000
40000 50000 50000
50000 60000 60000
60000 70000 70000
70000 80000 80000
80000 90000 90000
90000 100000 100000
100000 110000 110000
110000 120000 120000
120000 130000 130000
130000 140000 140000
140000 150000 150000
150000 160000 160000
160000 170000 170000


In [114]:
x = itertools.islice(dict_.items(), 170001,188595)
for key,val in x:
    count += 1
    if key not in sent:
        sent[key] = []
    try:
        sentiment = sentiment_analyzer.polarity_scores(val)
        sentiment.pop('compound')
        sentiment.pop('neu')
        sent[key].append(max(sentiment.items(), key=operator.itemgetter(1))[0])
    except:
        print(key, val)
        continue

# print(lower_limit, upper_limit, count)

1 nan
0 nan


In [ ]:
# x = itertools.islice(dict_.items(), 0,10000)

In [95]:
len(sent)

188595

In [96]:
sent

{'pomGBqfbxcqPv14c3XH-ZQ': ['neg'],
 'jtQARsP6P-LbkyjbO1qNGg': ['pos'],
 'elqbBhBfElMNSrjFqW3now': ['pos'],
 'Ums3gaP2qM3W1XcA5r6SsQ': ['pos'],
 'vgfcTvK81oD4r50NMjU2Ag': ['neg'],
 'AxeQEz3-s9_1TyIo-G7UQw': ['pos'],
 'zdE82PiD6wquvjYLyhOJNA': ['pos'],
 'EAwh1OmG6t6p3nRaZOW_AA': ['pos'],
 'atVh8viqTj-sqDJ35tAYVg': ['pos'],
 'yFumR3CWzpfvTH2FCthvVw': ['neg'],
 'UBv8heCQR0RPnUQG0zkXIQ': ['pos'],
 'hdgYnadxg0GANhWOJabr2g': ['pos'],
 'yEOu75XjwczngvWWlr0M_A': ['pos'],
 'f5O7v_X_jCg2itqacRfxhg': ['pos'],
 'XqNDr54eLDLRfZwo4l4dVA': ['neg'],
 '0bjFYstj8USMzEV4ZQldjA': ['pos'],
 'q0n4I-zqiI47xispOqc1lA': ['pos'],
 'alG1fb1kl2vmT8s34jbbHg': ['pos'],
 'gZGsReG0VeX4uKViHTB9EQ': ['pos'],
 '-bd4BQcl1ekgo7avaFngIw': ['pos'],
 'u73j2VQ3TGWdMO-AG7MABw': ['pos'],
 'VyVIneSU7XAWgMBllI6LnQ': ['pos'],
 '3Mx4renubPRnjHUw1n2UkA': ['pos'],
 'd4z4gjdhQYs-WOVClISf_A': ['pos'],
 'f-v1fvtnbdw_QQRsCnwH-g': ['pos'],
 'yz66FIUPDKGhILDWzRLeKg': ['neg'],
 '6qDRqHWqf0EeHupSUEBfKg': ['neg'],
 '7AlULGZI1pHt0imODsqdkg': [

In [ ]:
sent = {}
count = 0
for key,val in x:
#     print(val)
    sent[key] = []
    sentiment = sentiment_analyzer.polarity_scores(val)
#     print(sentiment)
    sentiment.pop('compound')
    sentiment.pop('neu')
    sent[key].append(max(sentiment.items(), key=operator.itemgetter(1))[0])
#     for s in sentiment:
#         print("{0}: {1} ".format(s, sentiment[s]), end='')


In [ ]:
len(sent)

In [ ]:
count

In [ ]:
sum(1 for x in sent.values() if x == 'pos')

In [ ]:
sum(1 for x in sent.values() if x == 'neg')

In [48]:
x = itertools.islice(dict_.items(), 10000,40000)

In [ ]:
for key,val in x:
    count +=1
    if key not in sent:
        sent[key] = []
    sentiment = sentiment_analyzer.polarity_scores(val)
#     print(sentiment)
    sentiment.pop('compound')
    sentiment.pop('neu')
    sent[key].append(max(sentiment.items(), key=operator.itemgetter(1))[0])
#     for s in sentiment:
#         print("{0}: {1} ".format(s, sentiment[s]), end='')

In [ ]:
len(sent)

In [ ]:
for key, val in x:
    print(key)

In [ ]:




x = itertools.islice(dict_.items(), 10000,40000)
for key,val in x:
    count +=1
    if key not in sent:
        sent[key] = []
    sentiment = sentiment_analyzer.polarity_scores(val)
#     print(sentiment)
    sentiment.pop('compound')
    sentiment.pop('neu')
    sent[key].append(max(sentiment.items(), key=operator.itemgetter(1))[0])
